## Stacking Classifier
El Stacking Classifier usa varios modelos base y luego utiliza un meta-modelo (también llamado modelo de segundo nivel) para hacer la predicción final. Este meta-modelo toma las predicciones de los modelos base como entrada.

## Cargar librería

In [8]:
import pandas as pd
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier


## Cargar dataset

In [4]:
# Ruta del archivo
ruta = r'C:/Users/Administrator/Documents/Proyecto seg clientes refuerzo/Proyecto_seg_clientes/Data/teleCust1000t_listo.csv'

# Cargar el dataset
df = pd.read_csv(ruta)

# Ver las primeras filas
print(df.head())
print(df.info())

   region  tenure  age  marital  address  income  ed  employ  retire  gender  \
0       2      13   44        1        9    64.0   4       5       0       0   
1       3      11   33        1        7   136.0   5       5       0       0   
2       3      68   52        1       24   116.0   1      29       0       1   
3       2      33   33        0       12    33.0   2       0       0       1   
4       2      23   30        1        9    30.0   1       2       0       0   

   reside  custcat  
0       2        1  
1       6        4  
2       2        3  
3       1        1  
4       4        3  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   region   1000 non-null   int64  
 1   tenure   1000 non-null   int64  
 2   age      1000 non-null   int64  
 3   marital  1000 non-null   int64  
 4   address  1000 non-null   int64  
 5   income   1000 no

## Preparar las características (X) y la variable objetivo (y)

In [6]:
# Separar características (X) y la variable objetivo (y)
X = df.drop(columns=['custcat'])  # Eliminar la columna objetivo
y = df['custcat']                # Variable objetivo

## Modelo stacking classifier

In [7]:
# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Modelos base
base_learners = [
    ('xgb', XGBClassifier(random_state=42)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('knn', KNeighborsClassifier(n_neighbors=5))
]

# Meta-modelo (en este caso, una regresión logística)
meta_model = LogisticRegression()

# Crear el Stacking Classifier
stacking_clf = StackingClassifier(estimators=base_learners, final_estimator=meta_model)

# Entrenar el Stacking Classifier
stacking_clf.fit(X_train_scaled, y_train)

# Hacer predicciones
y_pred = stacking_clf.predict(X_test_scaled)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Exactitud del modelo de Stacking Classifier: {accuracy:.2f}")

Exactitud del modelo de Stacking Classifier: 0.39


## Voting Classifier

In [9]:
# Crear los modelos base
model1 = XGBClassifier(random_state=42)
model2 = RandomForestClassifier(n_estimators=100, random_state=42)
model3 = KNeighborsClassifier(n_neighbors=5)

# Crear el Voting Classifier con votación 'hard' (voto mayoritario)
voting_clf = VotingClassifier(estimators=[
    ('xgb', model1), 
    ('rf', model2), 
    ('knn', model3)
], voting='hard')

# Entrenar el Voting Classifier
voting_clf.fit(X_train_scaled, y_train)

# Hacer predicciones
y_pred = voting_clf.predict(X_test_scaled)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Exactitud del modelo de Voting Classifier: {accuracy:.2f}")

Exactitud del modelo de Voting Classifier: 0.33
